In [2]:
import os
import json 
import copy
import numpy as np
import pandas as pd

import seaborn as sns
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from refpydst.evaluate_metrics import evaluate
from refpydst.utils.dialogue_state import update_dialogue_state
from refpydst.prompt_formats.python.completion_parser import parse_python_completion, iterative_parsing, parse_python_modified
from refpydst.normalization.data_ontology_normalizer import DataOntologyNormalizer
from refpydst.db.ontology import Ontology
import refpydst.prompt_formats.python.demo as python_demo
from refpydst.prompt_formats.python.completion_parser import *


pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
with open('../data/mw21_5p_train_v1.json', 'r') as f:
    train_data = json.load(f)
    
normalizer = DataOntologyNormalizer(
        Ontology.create_ontology(),
        # count labels from the train set
        supervised_set=train_data,
        # make use of existing surface form knowledge encoded in ontology.json, released with each dataset
        # see README.json within https://github.com/smartyfh/MultiWOZ2.4/raw/main/data/MULTIWOZ2.4.zip
        counts_from_ontology_file="../src/refpydst/db/multiwoz/2.4/ontology.json"
)

reading surface forms from ontology.json: 100%|██████████| 31/31 [00:03<00:00,  8.04it/s]


In [4]:
def sorted_dict(dict_a, by_key=True):
    if by_key:
        return dict(sorted(dict_a.items(), key=lambda item: item[0]))
    else:
        return dict(sorted(dict_a.items(), key=lambda item: item[1], reverse=True))

In [7]:
def collect_stats(experiment_folder_path, parsing_method='og_modified'):
    stats = pd.DataFrame()
    experiments = []
    for path, dir, files in os.walk(experiment_folder_path):
        if 'running_log.json' in files:
            name = path.split('/preliminary/')[-1]
            if name.endswith('running_log.json'):
                name = name.replace('running_log.json', '')
                continue

            log_path = os.path.join(path, 'running_log.json')
            # if "topk_bm_5_fs_5_0523_0315" not in exp_name:
            #     continue
            with open(log_path, 'r') as f:
                logs = json.load(f)
            
            jga_by_turn_id = defaultdict(list)  # use to record the accuracy
            jga_by_dialog = defaultdict(list)  # use to record the accuracy
            
            total_acc, total_f1 = 0, 0
            n_correct = 0
            n_total = len(logs)
            
            right, right_shots, right_logs = [], [], []
            wrong, wrong_shots, wrong_logs = [], [], []
            
            prior_pred, prior_id = None, None
            for data_item in logs:
                # pred = data_item['pred']
                if data_item.get('completion') is None:
                    n_correct += 1
                    tmp = []
                    for ex in data_item.get('examples', []):
                        tmp.append(ex[0].replace('.json', '_')+str(ex[1]))
                    data_item.update({'examples':tmp})
                    right_shots.append({
                        data_item['ID'].replace('.json', '_')+str(data_item['turn_id']):tmp})
                    right_logs.append({data_item['ID'].replace('.json', '_')+str(data_item['turn_id']):data_item})
                    right.append(data_item['ID'].replace('.json', '_')+str(data_item['turn_id']))

                    prior_id = data_item['ID']
                    prior_pred = data_item['pred']
                    # prior_pred_2 = data_item['pred']
                    continue
                
                if data_item['ID'] != prior_id:
                    prior_pred = data_item['pred_prior_context']
                    # prior_pred_2 = data_item['pred_prior_context']
                if parsing_method == 'og': 
                    pred = data_item['pred']
                elif parsing_method == 'og_modified':
                    pred_delta = normalizer.normalize(parse_python_modified(data_item['completion'], prior_pred))
                    pred =  update_dialogue_state(prior_pred, pred_delta)
                elif parsing_method == 'iterative':
                    pred_delta =  normalizer.normalize(iterative_parsing(data_item['completion'], prior_pred))
                    pred =  update_dialogue_state(prior_pred, pred_delta)

                this_jga, this_acc, this_f1 = evaluate(pred, data_item['slot_values'])
                total_acc += this_acc
                total_f1 += this_f1

                if this_jga:
                    n_correct += 1
        
                prior_id = data_item['ID']
                prior_pred = pred

            jga = n_correct / n_total
            slot_acc = total_acc/n_total
            slot_f1 = total_f1/n_total

            # if '0' in name.split('_')[-1] or '1' in name.split('_')[-1]:
            #     name = '_'.join(name.split('_')[:-2])
            retriever_input = 'None'
            if len(name.split('/')) >= 5:
                retriever_input = name.split('/')[4]
            stats = pd.concat([
                stats, pd.DataFrame({
                    'retriever': [name.split('/')[0]], 'prompt_format':[name.split('/')[1]], 'decoding':[name.split('/')[2]],
                    'model': [name.split('/')[3]], 'retriever_input': retriever_input, 'jga': [jga], 'right': n_correct, 'wrong': n_total-n_correct, 'total':n_total,
                    'slot_acc': [slot_acc], 'slot_f1': [slot_f1]})])
    # stats = stats.sort_values(by='jga', ascending=False).reset_index(drop=True)
    return stats

In [10]:
stats = collect_stats(experiment_folder_path="../outputs/runs/preliminary/bm25/plain_text_no_guidelines/greedy/8B", parsing_method='og')
stats.sort_values(by=['model','retriever_input', 'decoding'])

,retriever,prompt_format,decoding,model,retriever_input,jga,right,wrong,total,slot_acc,slot_f1
0,bm25,plain_text_no_guidelines,greedy,8B,context_bs,0.225985,327,1120,1447,0.921285,0.698195
0,bm25,plain_text_no_guidelines,greedy,8B,context_slot,0.226676,328,1119,1447,0.918959,0.677649
0,bm25,plain_text_no_guidelines,greedy,8B,context_text,0.217001,314,1133,1447,0.934094,0.764239
0,bm25,plain_text_no_guidelines,greedy,8B,dialog,0.478922,693,754,1447,0.966390,0.892936
0,bm25,plain_text_no_guidelines,greedy,8B,dialog_context_bs_0831_0258,0.461645,668,779,1447,0.965930,0.889750
0,bm25,plain_text_no_guidelines,greedy,8B,dialog_context_bs_gt_delta_bs_0904_2123,0.519696,752,695,1447,0.974660,0.914200
0,bm25,plain_text_no_guidelines,greedy,8B,dialog_context_bs_gt_delta_slot_0904_2145,0.494817,716,731,1447,0.970675,0.902619
0,bm25,plain_text_no_guidelines,greedy,8B,dialog_context_bs_gt_full_bs_0904_1903,0.548780,45,37,82,0.968699,0.908685
0,bm25,plain_text_no_guidelines,greedy,8B,dialog_context_bs_gt_full_bs_0904_2037,0.481686,697,750,1447,0.966183,0.890050
0,bm25,plain_text_no_guidelines,greedy,8B,dialog_context_bs_gt_full_slot_0904_2100,0.510712,739,708,1447,0.971021,0.901307


In [17]:
print(stats.loc[stats['model'] == '8B'])

  retriever             prompt_format decoding model       jga  right  wrong  \
0    random                plain_text   greedy    8B  0.332412    481    966   
0    random                plain_text     beam    8B  0.346925    502    945   
0    random                    python   greedy    8B  0.346925    502    945   
0    random                    python     beam    8B  0.369039    534    913   
0    random  plain_text_no_guidelines   greedy    8B  0.332412    481    966   
0    random  plain_text_no_guidelines     beam    8B  0.293020    424   1023   

   total  slot_acc   slot_f1  
0   1447  0.951209  0.854939  
0   1447  0.954181  0.857919  
0   1447  0.949850  0.842343  
0   1447  0.949666  0.847278  
0   1447  0.951209  0.854939  
0   1447  0.948100  0.821833  


In [33]:
# dialog_context_slot & plain_text
stat_8B = collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/bm25/8B", parsing_method='og')
print(stat_8B['jga'].mean(), '\n', stat_8B[::-1])
stat_8B[::-1]['jga']
stat_8B_py = collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/bm25/8B_python", parsing_method='og')
print()
print(stat_8B_py['jga'].mean(), '\n',stat_8B_py[::-1])

0.4321008984105045 
                   name       jga  right  wrong  total  slot_acc   slot_f1
0               dialog  0.432619    626    821   1447  0.962612  0.882085
0  dialog_context_slot  0.463027    670    777   1447  0.965930  0.890149
0    dialog_context_bs  0.456807    661    786   1447  0.965953  0.893026
0  dialog_context_text  0.375950    544    903   1447  0.955794  0.869709

0.418279198341396 
                   name       jga  right  wrong  total  slot_acc   slot_f1
0               dialog  0.446441    646    801   1447  0.957913  0.862942
0  dialog_context_slot  0.448514    649    798   1447  0.958443  0.860535
0    dialog_context_bs  0.438839    635    812   1447  0.961276  0.874511
0  dialog_context_text  0.339323    491    956   1447  0.945519  0.816673


In [32]:
stat_8B_no_g = collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/bm25/8B_no_guidelines", parsing_method='og')
print(stat_8B_no_g['jga'].mean(), '\n', stat_8B_no_g[::-1])

0.46026261230131305 
                    name       jga  right  wrong  total  slot_acc   slot_f1
0    text_no_guidelines  0.450587    652    795   1447  0.966528  0.891959
0  dialog_no_guidelines  0.480304    695    752   1447  0.966897  0.894770
0      bs_no_guidelines  0.450587    652    795   1447  0.966528  0.891959
0    slot_no_guidelines  0.459572    665    782   1447  0.966321  0.888585


In [8]:
collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/bm25/70B", parsing_method='og')

,name,jga,right,wrong,total,slot_acc,slot_f1
0,dialog_0829_0117,0.553559,801,646,1447,0.976641,0.922718
1,dialog_context_bs,0.547339,792,655,1447,0.976480,0.920896
2,dialog_context_slot,0.539046,780,667,1447,0.975628,0.919356
3,text_0828_2323,0.485833,703,744,1447,0.969615,0.904433


In [9]:
collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/bm25/70B_no_guidelines", parsing_method='og')

,name,jga,right,wrong,total,slot_acc,slot_f1
0,slot_no_guidelines,0.579820,839,608,1447,0.977586,0.919676
1,dialog_no_guidelines,0.567381,821,626,1447,0.976641,0.920179
2,text_no_guidelines,0.541120,783,664,1447,0.976434,0.920695
3,bs_no_guidelines,0.539737,781,666,1447,0.976111,0.919804


In [34]:
# dialog_context_slot & python
stats_8B = collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/pt_sbert/8B", parsing_method='og')
stat_8B_py = collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/pt_sbert/8B_python", parsing_method='og')
stats_8B[::-1], stat_8B_py[::-1]

(                  name       jga  right  wrong  total  slot_acc   slot_f1
 0               dialog  0.402903    583    864   1447  0.962405  0.883426
 0  dialog_context_slot  0.435384    630    817   1447  0.964916  0.890661
 0    dialog_context_bs  0.398065    576    871   1447  0.962820  0.884394
 0  dialog_context_text  0.386317    559    888   1447  0.953951  0.861409,
                   name       jga  right  wrong  total  slot_acc   slot_f1
 0               dialog  0.426399    617    830   1447  0.960332  0.873759
 0  dialog_context_slot  0.454734    658    789   1447  0.960839  0.869271
 0    dialog_context_bs  0.402903    583    864   1447  0.957982  0.864274
 0  dialog_context_text  0.368348    533    914   1447  0.947086  0.839383)

In [35]:
stats_8B_no_g = collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/pt_sbert/8B_no_guidelines", parsing_method='og')
stats_8B_no_g[::-1]

,name,jga,right,wrong,total,slot_acc,slot_f1
0,text_no_guidelines,0.342087,495,952,1447,0.951279,0.848809
0,dialog_no_guidelines,0.327574,474,973,1447,0.950818,0.846306
0,bs_no_guidelines,0.342087,495,952,1447,0.951279,0.848809
0,slot_no_guidelines,0.338632,490,957,1447,0.951854,0.848742


In [39]:
for jga in stats_8B_no_g[::-1]['jga']:
    print(jga)

0.34208707671043537
0.32757429163787144
0.34208707671043537
0.33863165169315823


In [12]:
collect_stats(experiment_folder_path="../outputs/runs/preliminary/retriever_input/pt_sbert/70B", parsing_method='og')

,name,jga,right,wrong,total,slot_acc,slot_f1
0,dialog_context_bs,0.529371,766,681,1447,0.973048,0.912124
1,slot_0829_0306,0.502419,727,720,1447,0.970629,0.904161
2,dialog,0.498963,722,725,1447,0.972241,0.906367
3,dialog_context_text,0.433310,627,820,1447,0.962267,0.884485


In [77]:
model = 'llama'
experiment_folder_path = '/home/haesungpyun/my_refpydst/outputs/runs/'
experiment_folder_path += 'preliminary/decoding'
sufix = '' if model == 'gpt' else '_llama'
experiment_folder_path += sufix
parsing_method = 'og_modified'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

stats = collect_stats(experiment_folder_path=experiment_folder_path, parsing_method=parsing_method)

In [4]:
def collect_stats(experiment_folder_path, parsing_method='og_modified'):
    stats_8B = pd.DataFrame()
    stats_70B = pd.DataFrame()
    # stats_gpt = pd.DataFrame()
    experiments = []
    for path, dir, files in os.walk(experiment_folder_path):
        if 'running_log.json' in files:
            name = path.split('/preliminary/')[-1]
            if name.endswith('running_log.json'):
                name = name.replace('running_log.json', '')
                continue
            name = name.replace('/', '_')
            log_path = os.path.join(path, 'running_log.json')
            # if "topk_bm_5_fs_5_0523_0315" not in exp_name:
            #     continue
            with open(log_path, 'r') as f:
                logs = json.load(f)
            
            jga_by_turn_id = defaultdict(list)  # use to record the accuracy
            jga_by_dialog = defaultdict(list)  # use to record the accuracy
            
            total_acc, total_f1 = 0, 0
            n_correct = 0
            n_total = len(logs)
            
            right, right_shots, right_logs = [], [], []
            wrong, wrong_shots, wrong_logs = [], [], []
            
            prior_pred, prior_id = None, None
            for data_item in logs:
                # pred = data_item['pred']
                if data_item.get('completion') is None:
                    n_correct += 1
                    tmp = []
                    for ex in data_item.get('examples', []):
                        tmp.append(ex[0].replace('.json', '_')+str(ex[1]))
                    data_item.update({'examples':tmp})
                    right_shots.append({
                        data_item['ID'].replace('.json', '_')+str(data_item['turn_id']):tmp})
                    right_logs.append({data_item['ID'].replace('.json', '_')+str(data_item['turn_id']):data_item})
                    right.append(data_item['ID'].replace('.json', '_')+str(data_item['turn_id']))

                    prior_id = data_item['ID']
                    prior_pred = data_item['pred']
                    # prior_pred_2 = data_item['pred']
                    continue
                
                if data_item['ID'] != prior_id:
                    prior_pred = data_item['pred_prior_context']
                    # prior_pred_2 = data_item['pred_prior_context']
                if parsing_method == 'og': 
                    pred = data_item['pred']
                elif parsing_method == 'og_modified':
                    pred_delta = normalizer.normalize(parse_python_modified(data_item['completion'], prior_pred))
                    pred =  update_dialogue_state(prior_pred, pred_delta)
                elif parsing_method == 'iterative':
                    pred_delta =  normalizer.normalize(iterative_parsing(data_item['completion'], prior_pred))
                    pred =  update_dialogue_state(prior_pred, pred_delta)

                this_jga, this_acc, this_f1 = evaluate(pred, data_item['slot_values'])
                total_acc += this_acc
                total_f1 += this_f1

                if this_jga:
                    n_correct += 1
        
                prior_id = data_item['ID']
                prior_pred = pred

            jga = n_correct / n_total
            slot_acc = total_acc/n_total
            slot_f1 = total_f1/n_total

            if '8B' in name:
                stats_8B = pd.concat([
                    stats_8B, pd.DataFrame({
                        'name': [name], 'jga': [jga], 'right': n_correct, 'wrong': n_total-n_correct, 
                        'slot_acc': [slot_acc], 'slot_f1': [slot_f1]})])    
            elif '70B' in name:
                stats_70B = pd.concat([
                    stats_70B, pd.DataFrame({
                        'name': [name], 'jga': [jga], 'right': n_correct, 'wrong': n_total-n_correct, 
                        'slot_acc': [slot_acc], 'slot_f1': [slot_f1]})])    
            # elif 'gpt' in name:
            #     stats_gpt = pd.concat([
            #         stats_gpt, pd.DataFrame({
            #             'name': [name], 'jga': [jga], 'right': n_correct, 'wrong': n_total-n_correct, 
            #             'slot_acc': [slot_acc], 'slot_f1': [slot_f1]})])
    # stats = stats.sort_values(by='jga', ascending=False).reset_index(drop=True)
    # Merge the stats with melt and pivot
    # stats = pd.pivot_table(pd.concat([stats_8B, stats_70B, stats_gpt]), index='name', values=['jga', 'right', 'wrong', 'slot_acc', 'slot_f1'])
    stats_8B = stats_8B.sort_values(by='jga', ascending=False).reset_index(drop=True)
    stats_70B = stats_70B.sort_values(by='jga', ascending=False).reset_index(drop=True)
    # stats_gpt = stats_gpt.sort_values(by='jga', ascending=False).reset_index(drop=True)

    return stats_8B, stats_70B, #stats_gpt

In [5]:
model = 'llama'
experiment_folder_path = '/home/haesungpyun/my_refpydst/outputs/runs/'
experiment_folder_path += 'preliminary/decoding'
sufix = '' if model == 'gpt' else '_llama'
experiment_folder_path += sufix
parsing_method = 'og_modified'

In [17]:
import warnings
warnings.filterwarnings('ignore')

stats_8B, stats_70B = collect_stats(experiment_folder_path='../outputs/runs/preliminary/retriever_input', parsing_method='og')

In [15]:
stats_new = copy.deepcopy(stats_8B)

In [19]:
stats_8B

,name,jga,right,wrong,slot_acc,slot_f1
0,retriever_input_bm25_8B_dialog_no_guidelines,0.480304,695,752,0.966897,0.894770
1,retriever_input_bm25_8B_dialog_context_slot,0.463027,670,777,0.965930,0.890149
2,retriever_input_bm25_8B_dialog_context_slot_no_guidelines,0.459572,665,782,0.966321,0.888585
3,retriever_input_bm25_8B_dialog_context_text_0828_1343,0.456807,661,786,0.965953,0.893026
4,retriever_input_bm25_8B_dialog_context_bs,0.456807,661,786,0.965953,0.893026
5,retriever_input_bm25_8B_dialog_context_text,0.456807,661,786,0.965953,0.893026
6,retriever_input_pt_sbert_8B_python_dialog_context_slot_0828_1302,0.454734,658,789,0.960839,0.869271
7,retriever_input_bm25_8B_dialog_context_text_no_guidelines,0.450587,652,795,0.966528,0.891959
8,retriever_input_bm25_8B_dialog_context_bs_no_guidelines,0.450587,652,795,0.966528,0.891959
9,retriever_input_bm25_8B_python_dialog_context_slot,0.448514,649,798,0.958443,0.860535


In [11]:
stats_70B

,name,jga,right,wrong,slot_acc,slot_f1
0,retriever_input_bm25_70B_dialog_context_slot_no_guidelines,0.579820,839,608,0.977586,0.919676
1,retriever_input_bm25_70B_dialog_no_guidelines,0.567381,821,626,0.976641,0.920179
2,retriever_input_bm25_70B_dialog_context_bs_0828_0237,0.547339,792,655,0.976480,0.920896
3,retriever_input_bm25_70B_dialog_context_text_no_guidelines,0.541120,783,664,0.976434,0.920695
4,retriever_input_bm25_70B_dialog_context_bs_no_guidelines,0.539737,781,666,0.976111,0.919804
5,retriever_input_pt_sbert_70B_dialog_context_bs,0.529371,766,681,0.973048,0.912124
6,retriever_input_pt_sbert_70B_dialog,0.498963,722,725,0.972241,0.906367


In [27]:
stats_8B

,name,jga,right,wrong,slot_acc,slot_f1
0,retriever_input_pt_sbert_8B_dialog_context_slot,0.435384,630,817,0.964916,0.890661
1,retriever_input_pt_sbert_8B_dialog_context_bs,0.398065,576,871,0.962820,0.884394
2,retriever_input_pt_sbert_8B_dialog_context_text,0.386317,559,888,0.953951,0.861409
3,retriever_input_pt_sbert_8B_dialog_context_bs_no_guidelines,0.342087,495,952,0.951279,0.848809
4,retriever_input_pt_sbert_8B_dialog_context_text_no_guidelines,0.342087,495,952,0.951279,0.848809
5,retriever_input_pt_sbert_8B_dialog_context_slot_no_guidelines,0.338632,490,957,0.951854,0.848742
6,retriever_input_pt_sbert_8B_dialog_no_guidelines,0.327574,474,973,0.950818,0.846306


In [28]:
stats_70B

,name,jga,right,wrong,slot_acc,slot_f1
0,retriever_input_pt_sbert_70B_dialog_context_bs,0.529371,766,681,0.973048,0.912124
1,retriever_input_pt_sbert_70B_dialog,0.498963,722,725,0.972241,0.906367


In [25]:
stats.to_csv(f'../stats{sufix}_{parsing_method}_parsing.csv', index=False, sep='\t')


In [30]:
stats

,name,jga,right,wrong,slot_acc,slot_f1
0,smapling_exp_topk_bm_5_fs_5_0620_1337,0.665515,963,484,0.700069,0.654518
1,mixed_retriever_fs_bm_sample_topk_70B_0710_0628,0.564616,817,630,0.973209,0.919344
2,bm25_10_all_sim_70B_0710_0037,0.563234,815,632,0.971988,0.918429
3,fine_tuned_sbert_70B_0710_0434,0.559088,809,638,0.972472,0.914667
4,mixed_retriever_fs_bm_topk_70B_0710_0822,0.548030,793,654,0.973025,0.917260
5,mixed_retriever_fs_bm_sum_topk_70B_0710_1026,0.544575,788,659,0.972495,0.916222
6,bm25_10_all_sim_div_70B_0710_0241,0.536973,777,670,0.968878,0.906766
7,fine_tuned_sbert_topk_70B_0709_0937,0.534209,773,674,0.971666,0.908936
8,pretrained_sbert_70B_0710_1413,0.524534,759,688,0.969247,0.908131
9,mixed_retriever_fs_bm_topk,0.498963,722,725,0.965999,0.893415


In [26]:
# 
stats_70b = stats.loc[stats['name'].str.contains('70B')]
stats_70b = stats_70b.loc[stats_70b['right'] + stats_70b['wrong'] > 1000]

In [27]:
stats_70b.reset_index(drop=True, inplace=True)

In [28]:
stats_70b.to_csv(f'../stats{sufix}_70B_{parsing_method}_parsing.csv', index=False, sep='\t')

In [29]:
stats_70b

,name,jga,right,wrong,slot_acc,slot_f1
0,mixed_retriever_fs_bm_sample_topk_70B_0710_0628,0.564616,817,630,0.973209,0.919344
1,bm25_10_all_sim_70B_0710_0037,0.563234,815,632,0.971988,0.918429
2,fine_tuned_sbert_70B_0710_0434,0.559088,809,638,0.972472,0.914667
3,mixed_retriever_fs_bm_topk_70B_0710_0822,0.548030,793,654,0.973025,0.917260
4,mixed_retriever_fs_bm_sum_topk_70B_0710_1026,0.544575,788,659,0.972495,0.916222
5,bm25_10_all_sim_div_70B_0710_0241,0.536973,777,670,0.968878,0.906766
6,fine_tuned_sbert_topk_70B_0709_0937,0.534209,773,674,0.971666,0.908936
7,pretrained_sbert_70B_0710_1413,0.524534,759,688,0.969247,0.908131
8,pretrained_sbert_topk_70B_0710_1220,0.484451,701,746,0.966459,0.903008
